# Array Computation

## The Slowness of Loops

Python's default implementation (known as CPython) does some operations very slowly.
This is in part due to the dynamic, interpreted nature of the language: the fact that types are flexible, so that sequences of operations cannot be compiled down to efficient machine code as in languages like C and Fortran.
Recently there have been various attempts to address this weakness: well-known examples are the [PyPy](http://pypy.org/) project, a just-in-time compiled implementation of Python; the [Cython](http://cython.org) project, which converts Python code to compilable C code; and the [Numba](http://numba.pydata.org/) project, which converts snippets of Python code to fast LLVM bytecode.
Each of these has its strengths and weaknesses, but it is safe to say that none of the three approaches has yet surpassed the reach and popularity of the standard CPython engine.

The relative sluggishness of Python generally manifests itself in situations where many small operations are being repeated – for instance looping over arrays to operate on each element.
For example, imagine we have an array of values and we'd like to compute the reciprocal of each.
A straightforward approach might look like this:

In [3]:
import numpy as np
np.random.seed(0)

def compute_reciprocals(values):
    output = np.empty(len(values))
    for i in range(len(values)):
        output[i] = 1.0 / values[i]
    return output

 #try to write more like belowe from now on       
values = np.random.randint(1, 10, size=5)
compute_reciprocals(values)

array([0.16666667, 1.        , 0.25      , 0.25      , 0.125     ])

This implementation probably feels fairly natural to someone from, say, a C or Java background.
But if we measure the execution time of this code for a large input, we see that this operation is very slow, perhaps surprisingly so!
We'll benchmark this with IPython's ``%timeit`` magic (discussed in [Profiling and Timing Code](01.07-Timing-and-Profiling.ipynb)):

In [4]:
big_array = np.random.randint(1, 100, size=1000000)
%timeit compute_reciprocals(big_array)

2 s ± 29.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


It takes several seconds to compute these million operations and to store the result!
When even cell phones have processing speeds measured in Giga-FLOPS (i.e., billions of numerical operations per second), this seems almost absurdly slow.
It turns out that the bottleneck here is not the operations themselves, but the type-checking and function dispatches that CPython must do at each cycle of the loop.
Each time the reciprocal is computed, Python first examines the object's type and does a dynamic lookup of the correct function to use for that type.
If we were working in compiled code instead, this type specification would be known before the code executes and the result could be computed much more efficiently.

## Introducing UFuncs

For many types of operations, NumPy provides a convenient interface into just this kind of statically typed, compiled routine. This is known as a *vectorized* operation.
This can be accomplished by simply performing an operation on the array, which will then be applied to each element.
This vectorized approach is designed to push the loop into the compiled layer that underlies NumPy, leading to much faster execution.

Compare the results of the following two:

In [5]:
print(compute_reciprocals(values))
print(1.0 / values)

[0.16666667 1.         0.25       0.25       0.125     ]
[0.16666667 1.         0.25       0.25       0.125     ]


In [6]:
%timeit (1.0 / big_array)

1.4 ms ± 92.4 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# Basic Functions

In [3]:
a = np.array((4, 3, 2, 1))

a.sort() # sorts the array in place
print(f"sorted: {a}")

sorted: [1 2 3 4]


In [4]:
a.sum()               # Sum
a.mean()              # Mean
a.max()               # Max
a.argmax()            # Returns the index of the maximal element
a.cumsum()            # Cumulative sum of the elements of a
a.cumprod()           # Cumulative product of the elements of a
a.var()               # Variance
a.std()               # Standard deviation

1.118033988749895

In [5]:
a.shape = (2, 2)#forces shape to be a 2 x 2 matrix
a

array([[1, 2],
       [3, 4]])

In [6]:
# Equivalent to a.transpose()
# flip around diagonal
a.T

array([[1, 3],
       [2, 4]])

If a is sorted `z.searchsorted(a)` returns the index of the first element of `z` that is `>= a`. It uses **binary search**

In [12]:
z = np.linspace(2, 4, 50) #linearly spaced numbers from 2-4 over 50 numbers
print(z)
z.searchsorted(2.2)#gives index where 2.2 is

[2.         2.04081633 2.08163265 2.12244898 2.16326531 2.20408163
 2.24489796 2.28571429 2.32653061 2.36734694 2.40816327 2.44897959
 2.48979592 2.53061224 2.57142857 2.6122449  2.65306122 2.69387755
 2.73469388 2.7755102  2.81632653 2.85714286 2.89795918 2.93877551
 2.97959184 3.02040816 3.06122449 3.10204082 3.14285714 3.18367347
 3.2244898  3.26530612 3.30612245 3.34693878 3.3877551  3.42857143
 3.46938776 3.51020408 3.55102041 3.59183673 3.63265306 3.67346939
 3.71428571 3.75510204 3.79591837 3.83673469 3.87755102 3.91836735
 3.95918367 4.        ]


5

# Aggregation

if we'd like to *reduce* an array with a particular operation, we can use the ``reduce`` method of any valid numpy function:

In [13]:
x = np.arange(1, 6)
print(x.sum())
# add.reduce is total sum
np.add.reduce(x)

15


ValueError: reduce only supported for binary functions

Similarly `accumulate` does this but keeps intermediate results

In [10]:
x = np.arange(1, 6)
print(x.cumsum())
# add.reduce is total sum
np.add.accumulate(x)

[ 1  3  6 10 15]


array([ 1,  3,  6, 10, 15])

And `outer` maps function into a pairwise table of results

In [11]:
np.multiply.outer(x, x) # makes cartesian map

array([[ 1,  2,  3,  4,  5],
       [ 2,  4,  6,  8, 10],
       [ 3,  6,  9, 12, 15],
       [ 4,  8, 12, 16, 20],
       [ 5, 10, 15, 20, 25]])

Since we are post-multiplying, the tuple is treated as a column vector.

# Filtering

The NumPy function `np.where` provides a vectorized array filtering:

In [22]:
x = np.random.randn(4)
print(x)
np.where(x > 0)  # Insert 1 if x > 0 true, otherwise 0

[ 0.78930459 -0.73209118 -0.81585044 -0.09495964]


(array([0]),)

This returns an array of indices where the condition is `True`. We can use it to index into other arrays:

In [23]:
x[np.where(x > 0)]

array([0.78930459])

### Bool Arrays

As a rule, comparisons on arrays are done element-wise and return an array of `bool`

In [26]:
z = np.array([2, 3])
y = np.array([2, 5])
z == y # returns boolean if vectors are same (2 = 2, 3 != 5)
#crashes if matrices aren't same size

array([ True, False])

We can also index into arrays from the bool array, to use as a filtering

In [27]:
y[z == y]

array([2])